In [1]:
import torch
%run /workspaces/jupyterlite/content/pytroch/arrays_and_matricies.py

### Algorithms

#### One-Dimensional Array

In [2]:
def fill_zero_places(a):
    """
    a = tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    b = tensor([-1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0, 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0])
    nonzero_indices = tensor([ 0,  1,  6,  8, 14, 23])
    diff = tensor([1, 5, 2, 6, 9, 8])
    c = tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    d = tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    """
    b = torch.cat([torch.tensor([-1], device=a.device), a])
    nonzero_indices = torch.nonzero(b).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([b.shape[0]], device=b.device)]))
    c = b[nonzero_indices].repeat_interleave(diff)
    c[c < 0] = 0
    d = c[1:]
    
    return d

In [3]:
def shift_values_to_next_available_zero(a):
    """
    a = tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    b = tensor([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    c = tensor([ 0,  1,  0,  0,  0,  0,  6,  0,  8,  0,  0,  0,  0,  0, 14,  0,  0,  0, 0,  0,  0,  0,  0, 23,  0,  0,  0,  0,  0,  0,  0])
    d = tensor([ 1,  0,  0,  0,  0,  6,  0,  8,  0,  0,  0,  0,  0, 14,  0,  0,  0,  0, 0,  0,  0,  0, 23,  0,  0,  0,  0,  0,  0,  0])
    e = tensor([ 0,  1,  1,  1,  1,  1,  6,  6,  8,  8,  8,  8,  8,  8, 14, 14, 14, 14, 14, 14, 14, 14, 14, 23, 23, 23, 23, 23, 23, 23])
    f = tensor([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    """
    b = (a > 0).int()
    b = torch.cat([torch.tensor([1], device=a.device), b])
    c = torch.arange(b.shape[0], device=a.device) * b
    d = c[1:]
    e = torch.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = torch.zeros_like(a)
    f.scatter_add_(0, e.long(), a)
        
    return f

In [4]:
def fill_zero_values_ascending_rest_with_zero(a):
    """
    a = tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 4, 3, 2, 1, 0, 0])
    b = tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0], dtype=torch.int32)
    c = tensor([0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 6, 7])
    d = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2])
    """
    b = (a > 0).int()
    c = torch.cumsum(1 - b, dim=0)
    d = (c * (1 - b) - torch.maximum(b * c, torch.zeros_like(c)).cummax(dim=0)[0]) * (1 - b)
    
    return d

In [5]:
def find_one_zero_pattern(a):
    """
    a = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
    b = tensor([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1], dtype=torch.int32)
    c = tensor([ 0,  0,  0, -1,  1, -1,  1,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0, 0,  0, -1,  1, -1,  1,  0,  0,  0,  0,  0, -1], dtype=torch.int32)
    d = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
    """
    b = torch.roll(a.int(), -1)
    b[-1] = 1
    c = a - b
    d = (c == 1).int()
    
    return d

In [6]:
def fill_zero_values_descending_rest_with_zero(a):
    """
    a = tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 4, 3, 2, 1, 0, 0])
    b = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2])
    c = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1], dtype=torch.int32)
    d = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
    e = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0])
    f = tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    g = tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    h = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0])
    """
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b > 0).int()
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g + 1) - b) * c
    
    return h

In [7]:
def find_values_that_must_be_reordered(a):
    """
    a = tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
    b = tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
    c = tensor([2, 8, 5, 7, 0, 7, 0, 6, 9, 5, 8, 8, 0, 7, 1, 8, 2, 6, 4, 2, 7, 0, 4, 2, 8, 6, 3, 4, 2, 6])
    d = tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 6])
    e = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1], dtype=torch.int32)
    f = tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0], dtype=torch.int32)
    """
    b = torch.arange(1, a.shape[0] + 1)
    c = a - b
    d, _ = torch.cummin(c.flip(0), dim=0)
    d = d.flip(0)
    e = (c == d).int()
    f = 1 - e

    return f

In [8]:
def adjust_cars_no_lane_change(a):
    """
    a = tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
    b = tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1], dtype=torch.int32)
    c = tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 4, 3, 2, 1, 0, 0])
    d = tensor([ 0,  0,  0,  0,  5,  0,  7,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0, 0,  0,  0, 22,  0, 26,  0,  0,  0,  0, 31, 36])
    e = tensor([ 5,  0,  0,  0,  0,  7,  0, 13,  0,  0,  0,  0,  0, 22,  0,  0,  0,  0, 0,  0,  0,  0, 26,  0, 31,  0,  0,  0,  0, 36])
    f = tensor([ 5,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 22, 22, 22, 22, 22, 22, 22, 22, 22, 26, 26, 31, 31, 31, 31, 31, 36])
    g = tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36])
    """
    b = 1 - find_values_that_must_be_reordered(a)
    c = fill_zero_values_descending_rest_with_zero(b)
    d = a * b
    e = shift_values_to_next_available_zero(d)
    f = fill_zero_places(e)
    g = f - c

    return g

#### Matrices

### Use-Cases

#### Single Lane

In [9]:
a = torch.tensor([ 83,  84,  89,  93,  87,  92,  90,  95,  90,  94,  94,  96,  97, 101,  97, 105, 104,  99, 101, 101, 101, 110, 105, 111, 106, 115, 90,  86, 107,  59])
adjust_cars_no_lane_change(a)

tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [10]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
adjust_cars_no_lane_change(a)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36])

In [11]:
a = torch.tensor([ 1,  4,  3,  6,  5,  6,  8,  8, 14, 10, 11, 13, 13, 14, 15, 16, 17, 18,  19, 20, 21, 23, 23, 24, 25, 26, 27, 28, 30, 35])
adjust_cars_no_lane_change(a)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 35])

#### Multi-Lane

In [12]:
# [lane, index, speed]
cars = torch.tensor([[0, 1, 4], [1, 5, 3], [0, 3, 5], [1, 7, 3], [1, 6, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 5, 5], [0, 6, 3]])

In [13]:
(cars[1:, 1] - cars[:-1, 1]  == 0).int()

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [14]:
def eligible_for_takeover(a, val_idx=1):
    """
    - two lane case
    - all cars with their original indicies (no adjustment to put them in one lane)
    - those cars where the index occurs twice cannot change the lane

    a = torch.tensor([[0, 1, 4], [0, 2, 3], [0, 3, 5], [1, 2, 3], [1, 3, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 9, 5], [0, 10, 3]], dtype=torch.float)
    b = tensor([0, 1, 0, 1, 0, 0, 0, 1, 0, 0], dtype=torch.int32)
    c = tensor([0, 1, 1, 1, 1, 0, 0, 1, 1, 0], dtype=torch.int32)
    d = tensor([0, 1, 1, 0, 1, 1, 1, 0, 1, 0], dtype=torch.int32)
    e = tensor([1, 0, 0, 1, 0, 0, 0, 1, 0, 1], dtype=torch.int32)
    """

    sorted_matrix = sort_matrix_by_nth_entry(a, val_idx)
    b = (sorted_matrix[:, val_idx] - torch.cat([sorted_matrix[1:, val_idx], torch.tensor([sorted_matrix[-2, val_idx]])]) == 0).int()
    c = torch.roll(b, 1) | b
    d = reverse_sort_matrix_by_nth_entry(a, c)
    e = 1 - d
    
    return e

In [15]:
a = torch.tensor([[0, 1, 4], [0, 2, 3], [0, 3, 5], [1, 2, 3], [1, 3, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 9, 5], [0, 10, 3]], dtype=torch.float)
eligible_for_takeover(a)

tensor([1, 0, 0, 1, 0, 0, 0, 1, 0, 1], dtype=torch.int32)

In [16]:
import torch

cars = torch.tensor([[0, 1, 4], [0, 2, 3], [0, 3, 5], [1, 2, 3], [1, 3, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 9, 5], [0, 10, 3]], dtype=torch.float)
max_values = maximum_value_by_grp(cars, 0, 1)

# before the updates, no car is eligible for takeover (all cars are already in order)
max_values_per_lane = maximum_value_by_grp(cars, 0, 1)
cars_in_single_lane = add_to_matrix_mapping_values(cars, max_values_per_lane)

# Update the cars
cars_update = torch.clone(cars_in_single_lane)
cars_update[:, 1] += cars_update[:, 2]

# Adjust the cars, such that they could fit into one lane (to execute adjust_cars_no_lane_change)
cars_update_ordered_by_grp = sort_matrix_by_nth_entry(cars_update)
cars_update_in_lane = torch.clone(cars_update_ordered_by_grp)
cars_update_in_lane[:, 1] = adjust_cars_no_lane_change(cars_update_ordered_by_grp[:, 1])[:]

# Create the original order of the cars for the updates
cars_updated_original_order = reverse_sort_matrix_by_nth_entry(cars_update, cars_update_in_lane)

cars_in_single_lane, cars_update, cars_update_in_lane, cars_updated_original_order

(tensor([[ 0., 11.,  4.],
         [ 0., 12.,  3.],
         [ 0., 13.,  5.],
         [ 1., 23.,  3.],
         [ 1., 24.,  7.],
         [ 0., 14.,  3.],
         [ 1., 31.,  1.],
         [ 1., 32.,  3.],
         [ 0., 19.,  5.],
         [ 0., 20.,  3.]]),
 tensor([[ 0., 15.,  4.],
         [ 0., 15.,  3.],
         [ 0., 18.,  5.],
         [ 1., 26.,  3.],
         [ 1., 31.,  7.],
         [ 0., 17.,  3.],
         [ 1., 32.,  1.],
         [ 1., 35.,  3.],
         [ 0., 24.,  5.],
         [ 0., 23.,  3.]]),
 tensor([[ 0., 14.,  4.],
         [ 0., 15.,  3.],
         [ 0., 16.,  5.],
         [ 0., 17.,  3.],
         [ 0., 22.,  5.],
         [ 0., 23.,  3.],
         [ 1., 26.,  3.],
         [ 1., 31.,  7.],
         [ 1., 32.,  1.],
         [ 1., 35.,  3.]]),
 tensor([[ 0., 14.,  4.],
         [ 0., 15.,  3.],
         [ 0., 16.,  5.],
         [ 1., 26.,  3.],
         [ 1., 31.,  7.],
         [ 0., 17.,  3.],
         [ 1., 32.,  1.],
         [ 1., 35.,  3.],
      

In [75]:
neg_max_values_per_lane = torch.clone(max_values_per_lane)
neg_max_values_per_lane[:, 1] = - neg_max_values_per_lane[:, 1]
cars_update_multi_lane = add_to_matrix_mapping_values(cars_update_in_lane, neg_max_values_per_lane)
cars_update_in_lane, cars_update_multi_lane

(tensor([[ 0.,  4.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  6.,  5.],
         [ 0.,  7.,  3.],
         [ 0., 12.,  5.],
         [ 0., 13.,  3.],
         [ 1.,  5.,  3.],
         [ 1., 10.,  7.],
         [ 1., 11.,  1.],
         [ 1., 14.,  3.]]),
 tensor([[ 0.,  4.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  6.,  5.],
         [ 0.,  7.,  3.],
         [ 0., 12.,  5.],
         [ 0., 13.,  3.],
         [ 1.,  5.,  3.],
         [ 1., 10.,  7.],
         [ 1., 11.,  1.],
         [ 1., 14.,  3.]]))

In [76]:
max_values_per_lane

tensor([[ 0., 10.],
        [ 1., 11.]])

In [19]:
updated_cars_to_be_reordered = find_values_that_must_be_reordered(cars_update_ordered_by_grp[:, 1])
original_cars_to_be_reordered = reverse_sort_matrix_by_nth_entry(cars_update, updated_cars_to_be_reordered)

updated_cars_to_be_reordered, original_cars_to_be_reordered

(tensor([1, 0, 1, 0, 1, 0, 0, 0, 0, 0], dtype=torch.int32),
 tensor([1, 0, 1, 0, 0, 0, 0, 0, 1, 0], dtype=torch.int32))

In [20]:
cars_update[:, 1]

tensor([15., 15., 18., 26., 31., 17., 32., 35., 24., 23.])

In [79]:
def mapping_max_values_to_prev_lanes(cars):
    """
    - two lanes
    """
    max_values_per_lane = maximum_value_by_grp(cars, 0, 1)
    mapping_max_values_per_lane = torch.clone(max_values_per_lane)
    mapping_max_values_per_lane[1:, 1] = mapping_max_values_per_lane[:-1, 1]
    mapping_max_values_per_lane[0, 1] = 0

    return mapping_max_values_per_lane

In [78]:
def cars_desiring_takeover(cars, grp_idx = 0, val_idx = 1, speed_idx = 2):
    """
    - two lane case
    - all cars with their original indicies (no adjustment to put them in one lane)

    returns:
     - original_cars_to_overtake: positions of cars (in their original index) which try to overtake
    """

    # before the updates, no car is eligible for takeover (all cars are already in order)
    mapping_max_values_per_lane = mapping_max_values_to_prev_lanes(cars)
    cars_in_single_lane = add_to_matrix_mapping_values(cars, mapping_max_values_per_lane)
    
    # Update the cars
    cars_update = torch.clone(cars_in_single_lane)
    cars_update[:, val_idx] += cars_update[:, speed_idx]
    
    # Adjust the cars, such that they could fit into one lane (to execute adjust_cars_no_lane_change)
    cars_update_ordered_by_grp = sort_matrix_by_nth_entry(cars_update)

    # Based on the cars in their respective groups, which are the ones that try to overtake?
    updated_cars_to_overtake = find_values_that_must_be_reordered(cars_update_ordered_by_grp[:, 1])
    original_cars_to_overtake = reverse_sort_matrix_by_nth_entry(cars_update, updated_cars_to_overtake)

    return original_cars_to_overtake

In [54]:
def cars_that_overtake(cars):
    cars_that_might_overtake = cars_desiring_takeover(cars) & eligible_for_takeover(cars)
    prob_that_cars_overtake = cars_that_might_overtake * cars[:, 3]
    cars_that_overtake = (prob_that_cars_overtake > torch.rand(prob_that_cars_overtake.shape[0]))

    return cars_that_overtake

In [80]:
def update_cars(cars):
    """
    - two car lanes
    """
    
    cars_that_change_lane = cars_that_overtake(cars)
    cars[:, 0] += cars_that_change_lane
    cars[:, 0] = torch.remainder(cars[:, 0], 2)

    # Put all cars in one lane
    max_values_per_lane = mapping_max_values_to_prev_lanes(cars)
    cars_in_single_lane = add_to_matrix_mapping_values(cars, max_values_per_lane)
    
    # Update the cars
    cars_update = torch.clone(cars_in_single_lane)
    cars_update[:, 1] += cars_update[:, 2]

    # Adjust the cars, such that they could fit into one lane (to execute adjust_cars_no_lane_change)
    cars_update_ordered_by_grp = sort_matrix_by_nth_entry(cars_update)
    cars_update_in_lane = torch.clone(cars_update_ordered_by_grp)
    cars_update_in_lane[:, 1] = adjust_cars_no_lane_change(cars_update_ordered_by_grp[:, 1])[:]

    # Remove the lane offset and put the cars to the original index
    neg_max_values_per_lane = torch.clone(max_values_per_lane)
    neg_max_values_per_lane[:, 1] = - neg_max_values_per_lane[:, 1]
    cars_update_multi_lane = add_to_matrix_mapping_values(cars_update_in_lane, neg_max_values_per_lane)

    return cars_update_multi_lane

In [56]:
import torch

cars = torch.tensor([[0, 1, 4], [0, 2, 3], [0, 3, 5], [1, 2, 3], [1, 3, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 9, 5], [0, 10, 3]], dtype=torch.float)
max_values = maximum_value_by_grp(cars, 0, 1)

# before the updates, no car is eligible for takeover (all cars are already in order)
max_values_per_lane = maximum_value_by_grp(cars, 0, 1)
cars_in_single_lane = add_to_matrix_mapping_values(cars, max_values_per_lane)

# Update the cars
cars_update = torch.clone(cars_in_single_lane)
cars_update[:, 1] += cars_update[:, 2]

# Adjust the cars, such that they could fit into one lane (to execute adjust_cars_no_lane_change)
cars_update_ordered_by_grp = sort_matrix_by_nth_entry(cars_update)
cars_update_in_lane = torch.clone(cars_update_ordered_by_grp)
cars_update_in_lane[:, 1] = adjust_cars_no_lane_change(cars_update_ordered_by_grp[:, 1])[:]

# Create the original order of the cars for the updates
cars_updated_original_order = reverse_sort_matrix_by_nth_entry(cars_update, cars_update_in_lane)


In [87]:
cars = torch.tensor([[0, 1, 4, 0.1], [0, 2, 3, 0.1], [0, 3, 5, 0.1], [1, 2, 3, 0.1], [1, 3, 7, 0.1], [0, 4, 3, 0.1], [1, 10, 1, 0.1], [1, 11, 3, 0.1], [0, 9, 5, 0.1], [0, 10, 3, 0.1]], dtype=torch.float)
cars = update_cars(cars)

tensor([[ 0.0000,  4.0000,  4.0000,  0.1000],
        [ 0.0000,  5.0000,  3.0000,  0.1000],
        [ 0.0000,  6.0000,  5.0000,  0.1000],
        [ 0.0000,  7.0000,  3.0000,  0.1000],
        [ 0.0000, 12.0000,  5.0000,  0.1000],
        [ 0.0000, 13.0000,  3.0000,  0.1000],
        [ 1.0000, 15.0000,  3.0000,  0.1000],
        [ 1.0000, 20.0000,  7.0000,  0.1000],
        [ 1.0000, 21.0000,  1.0000,  0.1000],
        [ 1.0000, 24.0000,  3.0000,  0.1000]])

In [74]:
import torch

cars = torch.tensor([[0, 1, 4, 0.1], [0, 2, 3, 0.1], [0, 3, 5, 0.1], [1, 2, 3, 0.1], [1, 3, 7, 0.1], [0, 4, 3, 0.1], [1, 10, 1, 0.1], [1, 11, 3, 0.1], [0, 9, 5, 0.1], [0, 10, 3, 0.1]], dtype=torch.float)
max_values = maximum_value_by_grp(cars, 0, 1)

cars_that_change_lane = cars_that_overtake(cars)
cars[:, 0] += cars_that_change_lane
cars[:, 0] = torch.remainder(cars[:, 0], 2)

cars

tensor([[ 1.0000,  1.0000,  4.0000,  1.0000],
        [ 0.0000,  2.0000,  3.0000,  0.1000],
        [ 0.0000,  3.0000,  5.0000,  0.1000],
        [ 1.0000, 12.0000,  3.0000,  0.1000],
        [ 1.0000, 13.0000,  7.0000,  0.1000],
        [ 0.0000,  4.0000,  3.0000,  0.1000],
        [ 1.0000, 20.0000,  1.0000,  0.1000],
        [ 1.0000, 21.0000,  3.0000,  0.1000],
        [ 0.0000,  9.0000,  5.0000,  0.1000],
        [ 0.0000, 10.0000,  3.0000,  0.1000]])

In [65]:
cars_that_change_lane

tensor([False, False, False, False, False, False, False, False, False, False])

In [52]:
cars_that_might_overtake = cars_desiring_takeover(cars) & eligible_for_takeover(cars)
prob_that_cars_overtake = cars_that_might_overtake * cars[:, 3]
cars_that_overtake = (prob_that_cars_overtake > torch.rand(prob_that_cars_overtake.shape[0]))
cars_that_overtake

tensor([False, False, False, False, False, False, False, False, False, False])

tensor([0.5393, 0.0325, 0.6074, 0.3039, 0.5253, 0.9845, 0.3841, 0.6454, 0.9784,
        0.2262])

In [24]:
import torch

cars = torch.tensor([[0, 1, 4], [0, 2, 3], [0, 3, 5], [1, 2, 3], [1, 3, 7], [0, 4, 3], [1, 10, 1], [1, 11, 3], [0, 9, 5], [0, 10, 3]], dtype=torch.float)
max_values = maximum_value_by_grp(cars, 0, 1)

# before the updates, no car is eligible for takeover (all cars are already in order)
max_values_per_lane = maximum_value_by_grp(cars, 0, 1)
mapping_max_values_per_lane = torch.clone(max_values_per_lane)
mapping_max_values_per_lane[1:, 1] = mapping_max_values_per_lane[:-1, 1]
mapping_max_values_per_lane[0, 1] = 0
cars_in_single_lane = add_to_matrix_mapping_values(cars, mapping_max_values_per_lane)

# Update the cars
cars_update = torch.clone(cars_in_single_lane)
cars_update[:, 1] += cars_update[:, 2]

# Adjust the cars, such that they could fit into one lane (to execute adjust_cars_no_lane_change)
cars_update_ordered_by_grp = sort_matrix_by_nth_entry(cars_update)
cars_update_in_lane = torch.clone(cars_update_ordered_by_grp)
cars_update_in_lane[:, 1] = adjust_cars_no_lane_change(cars_update_ordered_by_grp[:, 1])[:]

# Create the original order of the cars for the updates
cars_updated_original_order = reverse_sort_matrix_by_nth_entry(cars_update, cars_update_in_lane)

cars_in_single_lane, cars_update, cars_update_in_lane, cars_updated_original_order

(tensor([[ 0.,  1.,  4.],
         [ 0.,  2.,  3.],
         [ 0.,  3.,  5.],
         [ 1., 12.,  3.],
         [ 1., 13.,  7.],
         [ 0.,  4.,  3.],
         [ 1., 20.,  1.],
         [ 1., 21.,  3.],
         [ 0.,  9.,  5.],
         [ 0., 10.,  3.]]),
 tensor([[ 0.,  5.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  8.,  5.],
         [ 1., 15.,  3.],
         [ 1., 20.,  7.],
         [ 0.,  7.,  3.],
         [ 1., 21.,  1.],
         [ 1., 24.,  3.],
         [ 0., 14.,  5.],
         [ 0., 13.,  3.]]),
 tensor([[ 0.,  4.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  6.,  5.],
         [ 0.,  7.,  3.],
         [ 0., 12.,  5.],
         [ 0., 13.,  3.],
         [ 1., 15.,  3.],
         [ 1., 20.,  7.],
         [ 1., 21.,  1.],
         [ 1., 24.,  3.]]),
 tensor([[ 0.,  4.,  4.],
         [ 0.,  5.,  3.],
         [ 0.,  6.,  5.],
         [ 1., 15.,  3.],
         [ 1., 20.,  7.],
         [ 0.,  7.,  3.],
         [ 1., 21.,  1.],
         [ 1., 24.,  3.],
      

### Benchmarks

In [25]:
count = 30000
lane_num_tracks = torch.tensor([300000])
cars_position = torch.arange(0, count)
cars_speed = torch.randint(0,10, (count, 1)).flatten()

adjust_cars_no_lane_change(cars_position + cars_speed)

tensor([    0,     1,     2,  ..., 30000, 30001, 30002])

In [26]:
# import torch
# import time

# if not torch.cuda.is_available():
#     raise RuntimeError("CUDA is not available, please run on a CUDA-capable device.")

# iterations = 100000
# count = 30000

# lane_num_tracks = torch.tensor([300000000]).cuda()
# cars_position = torch.arange(0, count).cuda()
# cars_speed = torch.randint(0, 10, (count, 1)).flatten().cuda()

# def adjust_cars_no_lane_change(cars_info):
#     # Sample implementation, replace with the actual function
#     return cars_info * 2  # Example operation

# # Start timing
# start_time = time.time()

# # Execute the function multiple times
# for _ in range(iterations):
#     result = adjust_cars_no_lane_change(cars_position + cars_speed)

# # Optionally, synchronize and free the cache to get accurate timings
# torch.cuda.synchronize()
# torch.cuda.empty_cache()

# # End timing
# end_time = time.time()

# # Calculate elapsed time
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time} seconds")

Iterationen: 1.000.000
Autos: 300.000

Zeit: 16 Sekunden

Iterationen: 10.000.000
Autos: 30.000

Zeit: 94 Sekunden

Iterationen: 1.000.000
Autos: 3.000.000

Zeit: 178 Sekunden